<a href="https://colab.research.google.com/github/manojmanivannan/ApacheSparkEssentials/blob/master/SparkStreamByPluralsight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt-get install -y openjdk-8-jdk-headless scala jq  ssh > /dev/null
!if ! [[ -f "spark-2.3.1-bin-hadoop2.7.tgz" ]]; then wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz; tar xf spark-2.3.1-bin-hadoop2.7.tgz; fi
!if ! [[ -f "ngrok-stable-linux-amd64.zip" ]]; then wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip; unzip ngrok-stable-linux-amd64.zip; fi

!if ! [[ -f "goodls_linux_amd64" ]]; then wget -q https://github.com/tanaikech/goodls/releases/download/v2.0.1/goodls_linux_amd64; chmod +x goodls_linux_amd64; fi
!cat /etc/ssh/ssh_host_rsa_key.pub > /etc/ssh/authorized_keys
!apt-get install python3.7 python3-pip
!python3.7 -m pip install --upgrade pyspark findspark

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
0% [10 InRelease gpgv 15.9 kB]

In [ ]:
!export SPARK_HOME="/content/spark-2.3.1-bin-hadoop2.7"
!export PATH="$PATH:$SPARK_HOME/bin"

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"


In [ ]:
!/content/spark-2.3.1-bin-hadoop2.7/sbin/stop-master.sh
!/content/spark-2.3.1-bin-hadoop2.7/sbin/stop-slave.sh
!/content/spark-2.3.1-bin-hadoop2.7/sbin/start-master.sh

In [ ]:
!sleep 3 && grep -oP "MasterUI' on port ([0-9]+)" /content/spark-2.3.1-bin-hadoop2.7/logs/spark--org.apache.spark.deploy.master*.out 
!grep -oP "spark:.*" /content/spark-2.3.1-bin-hadoop2.7/logs/spark--org.apache.spark.deploy.master*.out 

In [ ]:
!/content/spark-2.3.1-bin-hadoop2.7/sbin/start-slave.sh $(grep -oP "spark:.*" /content/spark-2.3.1-bin-hadoop2.7/logs/spark--org.apache.spark.deploy.master*.out)
!sleep 3 && grep -oP "WorkerUI' on port ([0-9]+)" /content/spark-2.3.1-bin-hadoop2.7/logs/spark--org.apache.spark.deploy.worker*.out 

In [ ]:
get_ipython().system_raw('./ngrok authtoken 2IeJHLKYs6eB68PKJQM1VkpPu00_5JXQQZW3hofy6Q6vqTX2')
from time import sleep
def ngrok_tunnel_by_port(port: int):
  get_ipython().system_raw('pkill -f ngrok')
  get_ipython().system_raw(f'./ngrok http {port} &')
  sleep(4)
  print(get_ipython().getoutput('curl -s http://localhost:4040/api/tunnels',split=True))



In [ ]:
ngrok_tunnel_by_port(4041)

In [ ]:
!jps

In [ ]:
master_url = get_ipython().getoutput('grep -oP "spark:.*" /content/spark-2.3.1-bin-hadoop2.7/logs/spark--org.apache.spark.deploy.master*.out',split=True)
print('Master URL:',master_url[0])
# get_ipython().system_raw(f'nohup pyspark --master {master_url[0]} --num-executors 5 --driver-memory 4g --executor-memory 4g &' )

In [ ]:
!jps

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

# Spark session & context
conf = SparkConf().set("spark.master",master_url[0]).set('spark.ui.port', '4050').set("spark.executor.instances", 4).set("spark.executor.cores", 4).set("spark.driver.memory",'4g').set("spark.executor.memory",'4g')
try:
  sc = SparkContext(conf=conf)
except ValueError:
  sc.stop()
  sc = SparkContext(conf=conf)

spark = SparkSession.builder.appName('PluralSight').getOrCreate()

In [ ]:
ngrok_tunnel_by_port(4050)

In [ ]:
spark

In [ ]:
happiness_df = spark.read.format('csv').option('header','true').load('2016.csv')

In [ ]:
!head 2016.csv